# LookML for Refinement Layers


Stage One - Write the body code for each file.
Stage Two - Generate a full script for each refinement layer as well as the explore.
Stage Three - Generate the files with file names.
Stage Four - Fully automate the process of creating files in Github.


In [1]:
# Name this model
# i.e. datagrid.marketing_dm.impact_partner_performance_report
view_name = "marketing_aop_actuals_from_traffic_performance"
print(view_name)

marketing_aop_actuals_from_traffic_performance


In [2]:
# Paste in the dimensions from your raw lookML code.
text = """

  dimension: spend {
    description: ""
    value_format: "$#,##0"
    type: number
  }
  dimension: paid_search_spend {
    description: ""
    value_format: "$#,##0"
    type: number
  }
  dimension: social_spend {
    description: ""
    value_format: "$#,##0"
    type: number
  }
  dimension: marketplace_spend {
    description: ""
    value_format: "$#,##0"
    type: number
  }
  dimension: mcpa {
    label: "Traffic Performance MCPA"
    description: ""
    value_format: "$#,##0"
    type: number
  }
  dimension: sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: paid_search_sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: paid_social_sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: organic_plus_branded_search_sessions {
    label: "Traffic Performance O+BPS Sessions"
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: engaged_sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: paid_search_engaged_sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: paid_social_engaged_sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: engagement_rate {
    description: ""
    value_format: "0.00%"
    type: number
  }
  dimension: paid_search_engagement_rate {
    description: ""
    value_format: "0.00%"
    type: number
  }
  dimension: paid_social_engagement_rate {
    description: ""
    value_format: "0.00%"
    type: number
  }
  dimension: cost_per_engaged_session {
    label: "Traffic Performance CPES"
    description: ""
    value_format: "$#.00"
    type: number
  }
  dimension: paid_search_cost_per_engaged_session {
    label: "Traffic Performance Paid Search CPES"
    description: ""
    value_format: "$#.00"
    type: number
  }
  dimension: paid_social_cost_per_engaged_session {
    label: "Traffic Performance Paid Social CPES"
    description: ""
    value_format: "$#.00"
    type: number
  }
  dimension: click_thru_rate {
    label: "Traffic Performance CTR"
    description: ""
    value_format: "0.00%"
    type: number
  }
  dimension: click_thru_rate_branded_search {
    label: "Traffic Performance Branded Search CTR"
    description: ""
    value_format: "0.00%"
    type: number
  }
  dimension: click_thru_rate_non_branded_search {
    label: "Traffic Performance Non-Branded Search CTR"
    description: ""
    value_format: "0.00%"
    type: number
  }
  dimension: click_thru_rate_social_consideration {
    label: "Traffic Performance Social Consideration CTR"
    description: ""
    value_format: "0.00%"
    type: number
  }
  dimension: click_thru_rate_social_conversion {
    label: "Traffic Performance Social Conversion CTR"
    description: ""
    value_format: "0.00%"
    type: number
  }
  dimension: organic_search_sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: branded_search_sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: non_branded_search_sessions {
    label: "Traffic Performance Non-Branded Sessions"
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: email_sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: affiliate_sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: display_sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: paid_video_sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: direct_plus_other_sessions {
    label: "Traffic Performance Direct+Other Sessions"
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: organic_social_sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: non_email_sessions {
    label: "Traffic Performance Non-Email Sessions"
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: organic_plus_branded_search_spend {
    label: "Traffic Performance O+BPS Spend"
    description: ""
    value_format: "$#,##0"
    type: number
  }
  dimension: organic_search_mattress_orders {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: branded_search_mattress_orders {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: non_branded_search_mattress_orders {
    label: "Traffic Performance Non-Branded Search Mattress Orders"
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: affiliate_mattress_orders {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: direct_mattress_orders {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: email_mattress_orders {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: mattress_orders {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: organic_social_mattress_orders {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: paid_social_mattress_orders {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: paid_video_mattress_orders {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: display_social_mattress_orders {
    label: "Traffic Performance Display Mattress Orders"
    description: ""
    value_format: "#,##0"
    type: number
  }


"""

print(text)



  dimension: spend {
    description: ""
    value_format: "$#,##0"
    type: number
  }
  dimension: paid_search_spend {
    description: ""
    value_format: "$#,##0"
    type: number
  }
  dimension: social_spend {
    description: ""
    value_format: "$#,##0"
    type: number
  }
  dimension: marketplace_spend {
    description: ""
    value_format: "$#,##0"
    type: number
  }
  dimension: mcpa {
    label: "Traffic Performance MCPA"
    description: ""
    value_format: "$#,##0"
    type: number
  }
  dimension: sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: paid_search_sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: paid_social_sessions {
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: organic_plus_branded_search_sessions {
    label: "Traffic Performance O+BPS Sessions"
    description: ""
    value_format: "#,##0"
    type: number
  }
  dimension: engaged_

In [3]:
def parse_lookml(lookml_code):
    dimensions_string = []
    dimensions_number = []
    dimensions_time = []
    dimensions_boolean = []

    lines = lookml_code.split('\n')
    for line in lines:
        if line.strip().startswith('dimension'):
            dimension_name = line.split(':')[1].strip().split('{')[0].strip()  # Remove " {" at the end
            type_line = next(l for l in lines[lines.index(line):] if 'type:' in l)
            dimension_type = type_line.split(':')[1].strip()
            if dimension_type == 'string':
                dimensions_string.append(dimension_name)
            elif '_id' in dimension_name:
                dimensions_string.append(dimension_name)
            elif dimension_type == 'number':
                dimensions_number.append(dimension_name)
            elif dimension_type == 'time':
                dimensions_time.append(dimension_name)
            elif dimension_type == 'yesno':
                dimensions_boolean.append(dimension_name)

    return dimensions_string, dimensions_number, dimensions_time, dimensions_boolean


strings, numbers, times, booleans = parse_lookml(text)

print("String dimensions:", len(strings))
print("Boolean dimensions:", len(booleans))
print("Number dimensions:", len(numbers))
print("Time dimensions:", len(times))


String dimensions: 0
Boolean dimensions: 0
Number dimensions: 45
Time dimensions: 0


In [ ]:
strings

In [ ]:
booleans

In [ ]:
numbers

In [ ]:
times

In [4]:
# RENAME VIEW IN THE LOGIC LAYER
# create logic layer filename
print(view_name + '.logic')


marketing_aop_actuals_from_traffic_performance.logic


In [5]:
# ADD IMPORT (INCLUDE) HEADER FOR VIEW FILE
# create logical view file

def format_string(view_name):
    return f'''include: "/1_raw/{view_name}.view"

view: +{view_name} {{

}}'''

formatted_string = format_string(view_name)
print(formatted_string)

include: "/1_raw/marketing_aop_actuals_from_traffic_performance.view"

view: +marketing_aop_actuals_from_traffic_performance {

}


In [6]:
## Set Primary Key (optional if it is a date field)

def set_primary_key(list1, list2):
    looml = "## PRIMARY KEY\n\n"
    for item in list1 + list2:            
            if "sk" in item.lower():
                looml += f"dimension: {item} {{\n"
                looml += "    primary_key: yes\n"
                looml += "}\n"
                looml += "\n"
    return looml

# Example lists

result = set_primary_key(strings, numbers)
print(result)



## PRIMARY KEY




In [16]:
# create metrics
def generate_measure_lookml(names):
    lookml_code = '## BASIC METRICS\n\n'
    for name in names:
        lookml_code += f'measure: actuals_{name} {{\n'
        lookml_code += '  type: sum\n'
        lookml_code += f'  sql: ${{{name}}} ;;\n'
        lookml_code += '}\n\n'
    return lookml_code

generated_lookml = generate_measure_lookml(numbers)
print(generated_lookml)

## BASIC METRICS

measure: actuals_spend {
  type: sum
  sql: ${spend} ;;
}

measure: actuals_paid_search_spend {
  type: sum
  sql: ${paid_search_spend} ;;
}

measure: actuals_social_spend {
  type: sum
  sql: ${social_spend} ;;
}

measure: actuals_marketplace_spend {
  type: sum
  sql: ${marketplace_spend} ;;
}

measure: actuals_mcpa {
  type: sum
  sql: ${mcpa} ;;
}

measure: actuals_sessions {
  type: sum
  sql: ${sessions} ;;
}

measure: actuals_paid_search_sessions {
  type: sum
  sql: ${paid_search_sessions} ;;
}

measure: actuals_paid_social_sessions {
  type: sum
  sql: ${paid_social_sessions} ;;
}

measure: actuals_organic_plus_branded_search_sessions {
  type: sum
  sql: ${organic_plus_branded_search_sessions} ;;
}

measure: actuals_engaged_sessions {
  type: sum
  sql: ${engaged_sessions} ;;
}

measure: actuals_paid_search_engaged_sessions {
  type: sum
  sql: ${paid_search_engaged_sessions} ;;
}

measure: actuals_paid_social_engaged_sessions {
  type: sum
  sql: ${paid_soc

In [9]:
# CREATE STYLE LAYER FILE NAME

print(view_name + '.visual')

marketing_aop_actuals_from_traffic_performance.visual


In [10]:
# CREATE STYLE LAYER FILE HEADER

def format_string(view_name):
    return f'''include: "/3_logic_layer/{view_name}.logic.view"

view: +{view_name} {{

}}'''

formatted_string = format_string(view_name)
print(formatted_string)

include: "/3_logic_layer/marketing_aop_actuals_from_traffic_performance.logic.view"

view: +marketing_aop_actuals_from_traffic_performance {

}


In [11]:
## HIDE Primary Key & IDs

def hide_fields(list1, list2):
    looml = "\n\n## PRIMARY KEY & IDs \n\n"
    for item in list1 + list2:
        if "_id" in item.lower() or "sk" in item.lower() or "related_tranid" in item.lower():
            looml += f"dimension: {item} {{\n    hidden: yes\n"
            looml += "}\n"
            looml += "\n"
    return looml

# Example lists

result = hide_fields(strings, numbers)
print(result)





## PRIMARY KEY & IDs 




In [12]:
## HIDE Dimensions for Metrics

def hide_fields(list1):
    looml = "\n\n## HIDDEN DIMENSIONS for METRICS \n\n"
    for item in list1:
            looml += f"dimension: {item} {{\n    hidden: yes\n"
            looml += "}\n"
            looml += "\n"
    return looml

# Example lists

result = hide_fields(numbers)
print(result)



## HIDDEN DIMENSIONS for METRICS 

dimension: spend {
    hidden: yes
}

dimension: paid_search_spend {
    hidden: yes
}

dimension: social_spend {
    hidden: yes
}

dimension: marketplace_spend {
    hidden: yes
}

dimension: mcpa {
    hidden: yes
}

dimension: sessions {
    hidden: yes
}

dimension: paid_search_sessions {
    hidden: yes
}

dimension: paid_social_sessions {
    hidden: yes
}

dimension: organic_plus_branded_search_sessions {
    hidden: yes
}

dimension: engaged_sessions {
    hidden: yes
}

dimension: paid_search_engaged_sessions {
    hidden: yes
}

dimension: paid_social_engaged_sessions {
    hidden: yes
}

dimension: engagement_rate {
    hidden: yes
}

dimension: paid_search_engagement_rate {
    hidden: yes
}

dimension: paid_social_engagement_rate {
    hidden: yes
}

dimension: cost_per_engaged_session {
    hidden: yes
}

dimension: paid_search_cost_per_engaged_session {
    hidden: yes
}

dimension: paid_social_cost_per_engaged_session {
    hidden: 

In [17]:
# METRICS FORMATING
def generate_measure_lookml(names):
    lookml_code = '\n\n## METRICS\n\n'
    conditions = ['spend', 'revenue', 'cost', 'sales', 'asp', 'discount', 'roas']
    for name in names:
        # Set label and value format based on conditions

        if any(condition in name.lower() for condition in conditions):
            label = name.replace('_', ' ').title()  # Replace _ with space and make it proper case
            value_format = '"$#,##0"'
        else:
            label = name.replace('_', ' ').title()  # Replace _ with space and make it proper case
            value_format = '"#,##0"'

        lookml_code += f'measure: actuals_{name} {{\n'
        lookml_code += f'  label: "{label}"\n'
        lookml_code += f'  value_format: {value_format}\n'
        lookml_code += '}\n\n'
    return lookml_code

generated_lookml = generate_measure_lookml(numbers)
print(generated_lookml)



## METRICS

measure: actuals_spend {
  label: "Spend"
  value_format: "$#,##0"
  type: sum
  sql: ${spend} ;;
}

measure: actuals_paid_search_spend {
  label: "Paid Search Spend"
  value_format: "$#,##0"
  type: sum
  sql: ${paid_search_spend} ;;
}

measure: actuals_social_spend {
  label: "Social Spend"
  value_format: "$#,##0"
  type: sum
  sql: ${social_spend} ;;
}

measure: actuals_marketplace_spend {
  label: "Marketplace Spend"
  value_format: "$#,##0"
  type: sum
  sql: ${marketplace_spend} ;;
}

measure: actuals_mcpa {
  label: "Mcpa"
  value_format: "#,##0"
  type: sum
  sql: ${mcpa} ;;
}

measure: actuals_sessions {
  label: "Sessions"
  value_format: "#,##0"
  type: sum
  sql: ${sessions} ;;
}

measure: actuals_paid_search_sessions {
  label: "Paid Search Sessions"
  value_format: "#,##0"
  type: sum
  sql: ${paid_search_sessions} ;;
}

measure: actuals_paid_social_sessions {
  label: "Paid Social Sessions"
  value_format: "#,##0"
  type: sum
  sql: ${paid_social_sessions}

In [ ]:
## Date Formating

def generate_lookml(names):
    
    lookml_code = '\n\n## DATES & TIMES\n\n'
    
    for name in names:
        lookml_code += f'dimension_group: {name} {{\n'
        lookml_code += '  group_label: "Date"\n'
        lookml_code += '  label: ""\n'
        lookml_code += '}\n\n'
    return lookml_code

generated_lookml = generate_lookml(times)
print(generated_lookml)


In [ ]:
# create Explore file

print(view_name + '.explore')

In [ ]:
## Generate Explore lookML (optional)

lookML_code = ""
explore_label = ' '.join(word.capitalize() for word in view_name.split('_'))

lookML_code += f'include: "/3_visual/{view_name}.visual.view"\n\n'
lookML_code += f'explore: {view_name} {{\n'
lookML_code += f'   from: {view_name}\n'
lookML_code += f'   label: "{explore_label}"\n'
lookML_code += f'   group_label: "Marketing"\n'
lookML_code += f'}}'

print(lookML_code)

In [ ]:
## Commit Code

print(f'Deploy {explore_label} Explore into production.')